In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
import pandas as pd
train_path = '/kaggle/input/nlp-getting-started/train.csv'
df = pd.read_csv(train_path)
disaster=df[df['target']==1]['target'].value_counts()
non_disaster=df[df['target']==0]['target'].value_counts()
print(f'disaster count: {disaster}')
print(f'non_disaster count: {non_disaster}')
df['label'] = df['target']

disaster count: target
1    3271
Name: count, dtype: int64
non_disaster count: target
0    4342
Name: count, dtype: int64


In [3]:
df.head()

,id,keyword,location,text,target,label
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,1


In [4]:
from datasets import Dataset
data = Dataset.from_pandas(df[['text','label']])
data

Dataset({
    features: ['text', 'label'],
    num_rows: 7613
})

In [5]:
data[0]

{'text': 'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'label': 1}

In [6]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch

model_name_or_path = "Qwen/Qwen2-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path,device_map='auto',torch_dtype=torch.bfloat16,num_labels=2)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2-1.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 318.2 kB/s eta 0:00:00a 0:00:01


In [8]:
from peft import TaskType,get_peft_model,LoraConfig
lora_config = LoraConfig(
    task_type = TaskType.SEQ_CLS,
    r = 8,
    target_modules = ["q_proj", "k_proj"]
)
model = get_peft_model(model,lora_config)

In [9]:
model.config.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id

151645

In [10]:
len(df)

7613

In [11]:
dataset = data.train_test_split(test_size=0.2)
def preprocess_func(examples):
    return tokenizer(examples['text'],truncation=True,padding=True,return_tensors="pt")
encoded_dataset = dataset.map(preprocess_func,batched=True,batch_size=len(df))
encoded_dataset

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [12]:
from transformers import TrainingArguments,Trainer
num_train_epochs = 1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
warmup_steps = 10
weight_decay = 0.01
logging_steps = 1
save_path = '/output'
training_args = TrainingArguments(
    output_dir = save_path,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    per_device_eval_batch_size = per_device_eval_batch_size,
    warmup_steps = warmup_steps,
    weight_decay = weight_decay,
    logging_steps = logging_steps,
    gradient_accumulation_steps = 16,
    use_cpu=False,
    report_to="none"
)
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset = encoded_dataset['test']
)
trainer.train()

2024-08-13 08:02:01.095491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 08:02:01.095619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 08:02:01.225053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Step,Training Loss
1,1.933600
2,2.256300
3,2.109900
4,2.315200
5,2.364700
6,2.125700
7,2.185900
8,2.551800
9,2.021300
10,2.032400


TrainOutput(global_step=95, training_loss=1.8838090093512283, metrics={'train_runtime': 1256.2491, 'train_samples_per_second': 4.848, 'train_steps_per_second': 0.076, 'total_flos': 3970819250257920.0, 'train_loss': 1.8838090093512283, 'epoch': 0.9980302035456337})

In [13]:
test_path = '/kaggle/input/nlp-getting-started/test.csv'
df_test = pd.read_csv(test_path)
df_test_samples = df_test['text'].sample(10)
df_test_samples

2406    Refugees as citizens - The Hindu http://t.co/G...
134     @5SOStag honestly he could say an apocalypse i...
411     If you bored as shit don't nobody fuck wit you...
203     @RealTwanBrown Yesterday I Had A Heat Attack ?...
889     The Devil Wears Prada is still one of my favou...
1432    my father fucking died when the north tower co...
3024    Oh itÛªs a soccer ball? I thought it was the ...
2741    #Bestnaijamade: 16yr old PKK suicide bomber wh...
463        @ComplexMag he asking for a body bags @PUSHA_T
291     @JuneSnowpaw Yeah Gimme dat creamy white stuff ;3
Name: text, dtype: object

In [ ]:
for x in df_test_samples:
    inputs_ids = tokenizer([x],return_tensors="pt").to(model.device)
    outputs = model(**inputs_ids)
    predicted = outputs.logits.argmax(dim=-1).cpu().numpy().tolist()
    print(predicted)